# 时序差分（TD）学习方法原理笔记
## 一、TD 方法概述
时序差分（Temporal-Difference, TD）学习是强化学习中一类无模型（model-free） 的方法，核心是通过时序差分校正实现价值估计的迭代更新。它结合了蒙特卡洛（MC）方法的采样特性和动态规划（DP）的自举（bootstrapping）特性，能够在线更新价值估计，无需等待整个回合（episode）结束。
TD 方法的核心目标是估计状态价值（
v
）或动作价值（
q
），并通过价值估计优化策略，最终找到最优策略。
## 二、TD 方法的核心原理
1. 核心思想
TD 方法通过当前估计和下一时刻的估计之间的差异（时序差分）来更新价值，公式的通用形式为：
新估计=当前估计+步长×(目标−当前估计)
其中，“目标” 称为TD 目标（TD target），“目标与当前估计的差” 称为TD 误差（TD error）。
2. 数学基础
TD 方法本质是求解贝尔曼方程的随机近似算法（基于 Robbins-Monro 算法）。对于给定策略
π
，状态价值
v 
π
​
 (s)
满足贝尔曼期望方程：
v 
π
​
 (s)=E[R+γv 
π
​
 (S 
′
 )∣S=s]
TD 方法通过采样数据（
s,r,s 
′
 
）迭代逼近该方程的解，无需已知环境模型（状态转移概率
p(s 
′
 ∣s,a)
）。
3. 关键概念
自举（Bootstrapping）：使用当前价值估计（而非真实回报）更新价值，如利用
v(s 
′
 )
估计
v(s)
。
在线更新（Online Update）：每一步交互后立即更新价值，无需等待回合结束（区别于 MC 方法）。
TD 目标（TD Target）：用于校正当前估计的目标值，通常为即时奖励加折扣后的下一状态 / 动作价值估计。
TD 误差（TD Error）：当前估计与 TD 目标的差，反映估计的偏差，用于驱动更新。
## 三、主要 TD 算法
1. 估计状态价值的 TD 算法
原理
针对给定策略
π
，估计状态价值
v 
π
​
 (s)
，更新公式为：
v 
t+1
​
 (s 
t
​
 )=v 
t
​
 (s 
t
​
 )−α 
t
​
 (s 
t
​
 )⋅δ 
t
​
 
其中：
v 
t
​
 (s 
t
​
 )
：
t
时刻对
s 
t
​
 
的状态价值估计；
α 
t
​
 (s 
t
​
 )
：学习率（步长）；
TD 误差：
δ 
t
​
 =v 
t
​
 (s 
t
​
 )−[r 
t+1
​
 +γv 
t
​
 (s 
t+1
​
 )]
；
TD 目标：
v
ˉ
  
t
​
 =r 
t+1
​
 +γv 
t
​
 (s 
t+1
​
 )
（即时奖励加折扣后的下一状态价值估计）。
特点
仅更新当前访问状态
s 
t
​
 
的价值，未访问状态价值保持不变；
收敛性：当学习率满足
∑α 
t
​
 (s)=∞
且
∑α 
t
2
​
 (s)<∞
时，以概率 1 收敛到真实状态价值
v 
π
​
 (s)
。
2. Sarsa（估计动作价值）
原理
Sarsa 是用于估计动作价值（
q 
π
​
 (s,a)
）的 TD 算法，因使用数据
(s 
t
​
 ,a 
t
​
 ,r 
t+1
​
 ,s 
t+1
​
 ,a 
t+1
​
 )
而得名（State-Action-Reward-State-Action）。更新公式为：
q 
t+1
​
 (s 
t
​
 ,a 
t
​
 )=q 
t
​
 (s 
t
​
 ,a 
t
​
 )−α 
t
​
 (s 
t
​
 ,a 
t
​
 )⋅δ 
t
​
 
其中：
TD 误差：
δ 
t
​
 =q 
t
​
 (s 
t
​
 ,a 
t
​
 )−[r 
t+1
​
 +γq 
t
​
 (s 
t+1
​
 ,a 
t+1
​
 )]
；
TD 目标：
q
ˉ
​
  
t
​
 =r 
t+1
​
 +γq 
t
​
 (s 
t+1
​
 ,a 
t+1
​
 )
。
特点
On-policy：行为策略（生成数据的策略）与目标策略（待优化的策略）相同；
需结合策略改进（如
ϵ
- 贪心策略）实现最优策略搜索。
3. Expected Sarsa
原理
Sarsa 的变体，用期望代替下一个动作的采样，减少估计方差。更新公式为：
q 
t+1
​
 (s 
t
​
 ,a 
t
​
 )=q 
t
​
 (s 
t
​
 ,a 
t
​
 )−α 
t
​
 (s 
t
​
 ,a 
t
​
 )⋅δ 
t
​
 
其中TD 目标为：
q
ˉ
​
  
t
​
 =r 
t+1
​
 +γE[q 
t
​
 (s 
t+1
​
 ,A)∣s 
t+1
​
 ]=r 
t+1
​
 +γ∑ 
a
​
 π(a∣s 
t+1
​
 )q 
t
​
 (s 
t+1
​
 ,a)
特点
计算量大于 Sarsa（需计算所有可能下一个动作的期望），但方差更小；
仍为 On-policy 算法。
4. n-step Sarsa
原理
结合 Sarsa 和 MC 方法，通过n 步回报更新价值，平衡偏差和方差。n 步 TD 目标定义为：
G 
t
(n)
​
 =r 
t+1
​
 +γr 
t+2
​
 +⋯+γ 
n
 q 
t
​
 (s 
t+n
​
 ,a 
t+n
​
 )
更新公式为：
q 
t+n
​
 (s 
t
​
 ,a 
t
​
 )=q 
t+n−1
​
 (s 
t
​
 ,a 
t
​
 )−α 
t+n−1
​
 ⋅[q 
t+n−1
​
 (s 
t
​
 ,a 
t
​
 )−G 
t
(n)
​
 ]
特点
当
n=1
时退化为 Sarsa，当
n→∞
时接近 MC 方法；
n
越大，偏差越小但方差越大。
5. Q-learning
原理
直接估计最优动作价值（
q 
∗
​
 (s,a)
）的 Off-policy 算法，更新公式为：
q 
t+1
​
 (s 
t
​
 ,a 
t
​
 )=q 
t
​
 (s 
t
​
 ,a 
t
​
 )−α 
t
​
 (s 
t
​
 ,a 
t
​
 )⋅δ 
t
​
 
其中TD 目标为：
q
ˉ
​
  
t
​
 =r 
t+1
​
 +γmax 
a
​
 q 
t
​
 (s 
t+1
​
 ,a)
特点
Off-policy：行为策略（如
ϵ
- 贪心）与目标策略（贪心策略）可不同，灵活性更高；
直接求解贝尔曼最优方程：
q 
∗
​
 (s,a)=E[R+γmax 
a
​
 q 
∗
​
 (s 
′
 ,a)∣s,a]
；
收敛性：满足学习率条件时，收敛到最优动作价值
q 
∗
​
 (s,a)
。
## 四、TD 方法与其他方法的对比
特性	TD 方法	蒙特卡洛（MC）方法	动态规划（DP）方法
模型依赖	无（model-free）	无（model-free）	有（需状态转移概率）
更新时机	在线（每步后）	离线（回合结束后）	全状态同步更新
自举（Bootstrapping）	是	否	是
方差	低（依赖少量采样）	高（依赖整个回合）	无（基于模型）
偏差	有（依赖估计值）	无（基于真实回报）	无（基于模型）
## 五、统一视角
所有 TD 算法可表示为统一形式：
新估计=当前估计−α⋅(当前估计−TD目标)
差异仅在于TD 目标的定义：
算法	TD 目标（
q
ˉ
​
  
t
​
 
）
Sarsa	
r 
t+1
​
 +γq 
t
​
 (s 
t+1
​
 ,a 
t+1
​
 )
Expected Sarsa	
r 
t+1
​
 +γ∑ 
a
​
 π(a∣s 
t+1
​
 )q 
t
​
 (s 
t+1
​
 ,a)
n-step Sarsa	
r 
t+1
​
 +γr 
t+2
​
 +⋯+γ 
n
 q 
t
​
 (s 
t+n
​
 ,a 
t+n
​
 )
Q-learning	
r 
t+1
​
 +γmax 
a
​
 q 
t
​
 (s 
t+1
​
 ,a)
MC 方法	
r 
t+1
​
 +γr 
t+2
​
 +⋯
（无自举）
## 六、总结
TD 方法是强化学习中极具影响力的无模型方法，其核心是通过时序差分校正实现价值的在线更新。关键优势包括：
无需环境模型，适用于复杂场景；
在线更新，可处理持续任务（continuing tasks）；
方差低于 MC 方法，收敛更稳定。